TODO:
    
Decide on how to create 2010 geom table from 2000.
- We can create new geoid indices on 2000 tables to create a massive join
- We can ingest entire 01-37 tables individually into RAM in python and interpolate in ram in place and write to
     new 2000 tables
- For either of the above we can decide to write all results into a single massive table, including new geoid column


In [28]:
import array, csv, json, math, multiprocessing, numpy, os, random, re, shutil
import shapely, shapely.wkb, struct, subprocess, sys, tempfile, threading, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

set_default_psql_database('census2010')

set default_psql_database to census2010


Crosswalk:  download and document
---------------------------------

In [2]:
download_file('http://users.pop.umn.edu/~jps/NHGIS_block2000_to_block2010.zip', 'capture/NHGIS_block2000_to_block2010.zip')

capture/NHGIS_block2000_to_block2010.zip already downloaded


In [ ]:
unzip_file('capture/NHGIS_block2000_to_block2010.zip')

In [ ]:
!cat capture/NHGIS_block2000_to_block2010/readme.txt

In [4]:
!head capture/NHGIS_block2000_to_block2010/crosswalk_block2000_block2010_v002.csv

F010010201001000,F010010201002000,0.03589746619998647,0.008988216111782212
F010010201001000,F010010201002001,0.2533302636105429,0.2637247584498352
F010010201001000,F010010201002002,0,0.000384823707001651
F010010201001000,F010010201002003,0.0762971112558778,0.055429504205606606
F010010201001000,F010010201002004,0.03244112598630059,0.007542916917166581
F010010201001000,F010010201002005,0.41140573696128735,0.18976089744424943
F010010201001000,F010010201002006,0.019909297520890202,0.0019957785723459264
F010010201001000,F010010201002007,0.09924123825815177,0.45777176371172934
F010010201001000,F010010201002008,0.07147776020696295,0.014401340880283067
F010010201001001,F010010201002004,0.13459272780487186,0.04845679605124167


In [3]:
psql('SELECT * FROM crosswalk_block2000_block2010 '
    "WHERE geoid2010='010010201002007'")

SELECT * FROM crosswalk_block2000_block2010 WHERE geoid2010='010010201002007'
Finished execution in 0.185084 secs: geoid2000    |    geoid2010    |  weight   |  parea   
-----------------+-----------------+-----------+----------
 010010201001000 | 010010201002007 | 0.0992412 | 0.457772
(1 row)


Crosswalk columns are:
block_id_2000, block_id_2010, weight, parea
We'll tentatively ignore the last.

For a data column X, we want to compute:
x2k(block_id_2010) = sum(x2k(block_id_2000)*weight(block_id_2000, block_id_2010)) for all block_id_2000 overlapping block_id_2010



Create crosswalk_block2000_block2010 database table
------------------------------------

In [ ]:
psql('CREATE TABLE crosswalk_block2000_block2010 '
    '(geoid2000 varchar, '
    ' geoid2010 varchar, '
    ' weight real, '
    ' parea real)')

In [ ]:
full_path = os.path.abspath('capture/NHGIS_block2000_to_block2010/crosswalk_block2000_block2010_v002.csv')
psql("COPY crosswalk_block2000_block2010 "
     "FROM '%s' DELIMITER ',' CSV;" % full_path,
     database='census2010')

In [ ]:
psql("UPDATE crosswalk_block2000_block2010 SET "
     "geoid2000 = right(geoid2000, 15), "
     "geoid2010 = right(geoid2010, 15)",
     database='census2010')

In [ ]:
psql("CREATE INDEX ON crosswalk_block2000_block2010 (geoid2010)",
     database='census2010')

In [19]:
psql("CREATE INDEX ON crosswalk_block2000_block2010 (geoid2000)",
     database='census2010')

CREATE INDEX ON crosswalk_block2000_block2010 (geoid2000)
Finished execution in 179.913 secs: CREATE INDEX


In [ ]:
psql('\d crosswalk_block2000_block2010', database='census2010')


Create interpolated tables
--------------------------

In [73]:
psql('\d census2010_block_idxs')

\d census2010_block_idxs
Finished execution in 0.229947 secs: Table "public.census2010_block_idxs"
    Column    |       Type        |                                  Modifiers                                   
--------------+-------------------+------------------------------------------------------------------------------
 blockidx2010 | integer           | not null default nextval('census2010_block_idxs_blockidx2010_seq'::regclass)
 geoid2010    | character varying | 
Indexes:
    "census2010_block_idxs_pkey" PRIMARY KEY, btree (blockidx2010)
    "census2010_block_idxs_geoid2010_idx" UNIQUE, btree (geoid2010)


In [32]:
# Old code for creating a new interpolated table
#DROP TABLE IF EXISTS {new_table};
#DROP TABLE IF EXISTS {tmp_table};
#CREATE TABLE {tmp_table} (blockidx2010, geoid2010, {columns})
#AS SELECT MIN(blockidx2010), geoid2010, {sums}
#FROM crosswalk_block2000_block2010
#JOIN {old_table} USING (geoid2000)
#JOIN census2010_block_idxs USING (geoid2010)
#GROUP BY geoid2010;
#CREATE UNIQUE INDEX ON {tmp_table} (blockidx2010);
#CREATE UNIQUE INDEX ON {tmp_table} (geoid2010);
#ALTER TABLE {tmp_table} RENAME TO {new_table};


# census2010_block2010
# census2000_block2010

def interpolate_2000_to_2010(old_table, column, force=False):
    # old_table: sf1_2000_block_p001
    # census_table_name: p001
    # SUM(weight * p001001)
    dataset = 'census2000_block2010'
    
    cache_dir = 'columncache'
    
    # dataset, e.g. census2010 or census2000_int2010 or LEHD 2011
    # table within dataset, e.g. P001 for census, HAC for LEHD
    # column within table
    # {cache_dir}/{dataset}/{census_table}-{column}.numpy
    
    dir_name = '{cache_dir}/{dataset}'.format(**locals())
    !mkdir -p $dir_name
    
    cache_filename = '{dir_name}/{column}.numpy'.format(**locals())
    if os.path.exists(cache_filename) and not force:
        sys.stdout.write('{cache_filename} already exists\n'.format(**locals()))
        return

    query = """
SELECT SUM(weight * {column})::REAL
FROM crosswalk_block2000_block2010
JOIN {old_table} USING (geoid2000)
GROUP BY geoid2010
ORDER BY geoid2010;
""".format(**locals())

    data = numpy.array([0] + [x[0] for x in query_psql(query)],
                       dtype=numpy.float32)
    
    numpy_atomic_save(cache_filename, data)

# interpolate_2000_to_2010('sf1_2000_block_p001', 'p001001')

Execution of SELECT SUM(weight * p001001)::REAL
FROM crosswalk_block2000_block2010
JOIN sf1_2000_block_p001 USING (geoid2000)
GROUP BY geoid2010
ORDER BY geoid2010;
took 127.782 seconds and returned 11078297 rows
Wrote 11078298 records to columncache/census2000_block2010/p001001.numpy


In [ ]:
pool = SimpleThreadPoolExecutor(8)  # seems good for 64GB RAM earthserve2

for table in sorted(get_table_names('sf1_2000_block_%')):
    for column in get_census_column_names_from_view(table):
        pool.submit(interpolate_2000_to_2010, table, column)

pool.shutdown()
None

Execution of SELECT table_name FROM INFORMATION_SCHEMA.tables WHERE table_schema='public' AND table_type='BASE TABLE' AND table_name LIKE 'sf1_2000_block_%'
took 0.014595 seconds and returned 286 rows
Execution of SELECT column_name FROM information_schema.columns WHERE table_name='sf1_2000_block_h001' AND column_name LIKE 'h001%'
took 0.0153351 seconds and returned 1 rows
Execution of SELECT column_name FROM information_schema.columns WHERE table_name='sf1_2000_block_h002' AND column_name LIKE 'h002%'
took 0.0142982 seconds and returned 6 rows
Execution of SELECT column_name FROM information_schema.columns WHERE table_name='sf1_2000_block_h003' AND column_name LIKE 'h003%'
took 0.0157449 seconds and returned 3 rows
Execution of SELECT column_name FROM information_schema.columns WHERE table_name='sf1_2000_block_h004' AND column_name LIKE 'h004%'
took 0.010035 seconds and returned 3 rows
Execution of SELECT column_name FROM information_schema.columns WHERE table_name='sf1_2000_block_h00

Sanity-checking first interpolated table
----------------------------------------
The 2000 census includes Puerto Rico (FIPS 72), but the crosswalk doesn't, so checking that the overall populations match requires filtering out Puerto Rico from the original census count.

In [14]:
psql('\d sf1_2000_int2010_p001')
psql('SELECT * FROM sf1_2000_int2010_p001 LIMIT 5;')
psql('select SUM(p001001) from sf1_2000_int2010_p001;')
psql('select SUM(p001001) from sf1_2000_block_p001;')
psql("select SUM(p001001) from sf1_2000_block_p001 WHERE LEFT(geoid2000,2) != '72';")

\d sf1_2000_int2010_p001
Finished execution in 0.33153 secs: Table "public.sf1_2000_int2010_p001"
    Column    |       Type        | Modifiers 
--------------+-------------------+-----------
 blockidx2010 | integer           | 
 geoid2010    | character varying | 
 p001001      | double precision  | 
Indexes:
    "sf1_2000_int2010_p001_tmp_blockidx2010_idx" UNIQUE, btree (blockidx2010)
    "sf1_2000_int2010_p001_tmp_geoid2010_idx" UNIQUE, btree (geoid2010)
SELECT * FROM sf1_2000_int2010_p001 LIMIT 5;
Finished execution in 0.108088 secs: blockidx2010 |    geoid2010    |     p001001      
--------------+-----------------+------------------
            1 | 010010201001000 |               75
            2 | 010010201001001 |                0
            3 | 010010201001002 |                4
            4 | 010010201001003 | 57.0367555618286
            5 | 010010201001004 |                0
(5 rows)
select SUM(p001001) from sf1_2000_int2010_p001;
Finished execution in 1.99111 secs: sum  

In [19]:
psql('SELECT MAX(p001001) FROM sf1_2000_int2010_p001')
psql('SELECT MAX(p001001) FROM sf1_2000_block_p001')
psql('SELECT MAX(p001001) FROM sf1_2010_block_p001')


SELECT MAX(p001001) FROM sf1_2000_int2010_p001
Finished execution in 2.07472 secs:
max        
------------------
 9887.55843400955
(1 row)
SELECT MAX(p001001) FROM sf1_2000_block_p001
Finished execution in 1.79416 secs:
max  
-------
 23373
(1 row)
SELECT MAX(p001001) FROM sf1_2010_block_p001
Finished execution in 2.24097 secs:
max  
-------
 19352
(1 row)


Add geoid index to geo2000 table
--------------------------------

In [ ]:
#in hindsight, should have called this geoid2000
psql("ALTER TABLE geo2000 ADD COLUMN geoid CHARACTER(15)",
     database='census2010')

In [ ]:
psql("UPDATE geo2000 SET geoid ="
     " (state || county || tract || block)",
     database='census2010')

In [ ]:
psql("CREATE INDEX ON geo2000 (geoid)",
     database='census2010')

In [ ]:
psql("SELECT geoid FROM geo2000 WHERE sumlev='101' LIMIT 10", database='census2010')

In [ ]:
# Show some 2000 'geoids'
psql("SELECT state || county || tract || block FROM geo2000 WHERE sumlev='101' LIMIT 10", database='census2010')

File identification (FILEID),state/US abbreviation (STUSAB),summary levels (SUMLEV),and the
geographic component codes (GEOCOMP) are critical elements in identifying the geographic level
for each record. The STUSAB field identifies the highest level of geography for the file. In the case
of state file,it identifies the individual state. For SF 1 files,the following FILEID and STUSAB codes
are used:
SF 1 state and state equivalent files ‘uSF1’ ‘AL-WY’
SF 1 advance national file ‘uSF1A’ ‘US’
SF 1 final national file ‘uSF1F’ ‘US’